In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

def inrixAufA81(WaypointsDirectory, OutputDirectory,a81_path):
    waypoints_dir = WaypointsDirectory
    output_dir = OutputDirectory
    dias = ['04_01', '04_02', '04_03', '04_04', '04_05', '04_06', '04_07']  # Extend 

    chunk_size = 10000
    
    for dia in dias:
        input_csv = os.path.join(waypoints_dir, f'{dia}_DEU_waypoints.csv')
        output_csv = os.path.join(output_dir, f'LKWaufA81_{dia}.csv')
        header_written = False  
        
        found_trip_ids = set()
        
        for chunk_number, chunk in enumerate(pd.read_csv(input_csv, chunksize=chunk_size)):
            print(f"Processing Chunk {chunk_number + 1} for day {dia}")
            
           
            chunk.rename(columns={
                chunk.columns[0]: "Trip-Id",
                chunk.columns[1]: "WaypointSequence",
                chunk.columns[2]: "CaptureDate",
                chunk.columns[3]: "Latitude",
                chunk.columns[4]: "Longitude",
                chunk.columns[5]: "SegmentID",
                chunk.columns[6]: "ZoneName",
                chunk.columns[7]: "FRC",
                chunk.columns[8]: "DeviceId",
                chunk.columns[9]: "RawSpeed",
                chunk.columns[10]: "RawSpeedMetric",
                chunk.columns[11]: "CustomAttrs"
            }, inplace=True)
            
            
            geometry = [Point(xy) for xy in zip(chunk['Longitude'], chunk['Latitude'])]
            chunk_gdf = gpd.GeoDataFrame(chunk, geometry=geometry, crs='EPSG:4326').to_crs('EPSG:25832')
            
            
            joined = gpd.sjoin(chunk_gdf, a81_gdf, how='inner', predicate='intersects')
            
            if not joined.empty:
                print(f"LKWs auf A81 in chunk {chunk_number + 1} für den Tag {dia}")
                
                new_trip_ids = set(joined['Trip-Id']) - found_trip_ids  # Filter new Trip-Ids
                if new_trip_ids:
                    found_trip_ids.update(new_trip_ids)
                    
                    filtered_chunk = chunk[chunk['Trip-Id'].isin(new_trip_ids)]
                    
                    
                    if not header_written:
                        filtered_chunk.to_csv(output_csv, index=False, mode='w')
                        header_written = True
                    else:
                        filtered_chunk.to_csv(output_csv, index=False, header=False, mode='a')
                else:
                    print("All Trip-Ids from this chunk already processed.")
            else:
                print("Keine LWKs auf A81")
        
        print(f"Processing for day {dia} completed. Results saved in {output_csv}")
        
    print("Alle Daten wurden verarbeitet")


In [21]:
import pandas as pd

def UnbewirtschaftAutobahnArray():
    # Cargar el archivo Excel y establecer el encabezado en la segunda fila
    df = pd.read_excel('U:\\Desktop\\kmStand\\csv-daten\\Standortliste_Lkw-Ladenetz 1.xlsx', header=1)
    
    # Filtrar las filas donde la columna 'Bewirtschaftung' es 'Unbewirtschaftet'
    unbewirtschaft_df = df[df['Bewirtschaftung'] == 'Unbewirtschaftet']
    
    # Extraer las Autobahnen 'Unbewirtschaftet' en una lista única
    unbewirtschaftete_autobahnen = unbewirtschaft_df['Bundes-autobahn'].unique().tolist()
    
    # Guardar la información completa de 'Unbewirtschaftet' en un archivo CSV
    unbewirtschaft_df.to_csv('U:\\Desktop\\kmStand\\csv-daten\\rastatteUnbewirtschaft.csv', index=False)
    
    # Mostrar el resultado
    print("Unbewirtschaftete Autobahnen:", unbewirtschaftete_autobahnen)

    return unbewirtschaftete_autobahnen

# Llamada a la función
unbewirtschaftete_autobahnen = UnbewirtschaftAutobahnArray()


Unbewirtschaftete Autobahnen: ['A7', 'A43', 'A24', 'A60', 'A8', 'A70', 'A9', 'A27', 'A13', 'A6', 'A66', 'A31', 'A29', 'A64', 'A3', 'A40', 'A23', 'A2', 'A1', 'A11', 'A81', 'A49', 'A63', 'A14', 'A4', 'A44', 'A5', 'A92', 'A19', 'A72', 'A21', 'A15', 'A45', 'A39', 'A96', 'A10', 'A93', 'A95', 'A20', 'A38', 'A71', 'A17', 'A73', 'A36', 'A46', 'A94']


In [26]:
df = pd.read_csv('U:\\Desktop\\kmStand\\csv-daten\\rastatteUnbewirtschaft.csv')
df

,ID,Kennung,Name Rastanlage,Bewirtschaftung,Bundes-autobahn,Lkw-Netzanschlussleistung [kVA],Punktgeometrie,Netzbetreiber Nieder-/ Mittelspannung,Netzbetreiber Hochspannung,ID ggü. liegender Standort\n(<1000 m)
0,10,DE-BY-000264,Rottachtal-West,Unbewirtschaftet,A7,2200,POINT (4352301.821520938 2727901.396672888),AllgäuNetz GmbH & Co. KG,LEW Verteilnetz GmbH,153.0
1,12,DE-NW-000195,Karthaus (West),Unbewirtschaftet,A43,7920,POINT (4138477.309109099 3200088.999963156),Stadtwerke Dülmen GmbH,Westnetz GmbH,NaN
2,13,DE-BB-283901,Blesenberg,Unbewirtschaftet,A24,4840,POINT (4473042.276667711 3345630.4299998633),Stadtwerke Pritzwalk GmbH,e.dis Netz GmbH,105.0
3,14,DE-RP-000074,Steinebrück Ost,Unbewirtschaftet,A60,2200,POINT (4048971.55261934 3020354.3140952536),Westnetz GmbH,Westnetz GmbH,NaN
4,19,DE-SL-000520,Weiler,Unbewirtschaftet,A8,3080,POINT (4072253.5964365117 2932551.0283145),Netzwerke Merzig GmbH,VSE Verteilnetz GmbH,NaN
...,...,...,...,...,...,...,...,...,...,...
125,340,DE-NW-007399,Alfert,Unbewirtschaftet,A46,2200,POINT (4210008.834437138 3141254.34378354),Westnetz GmbH,Westnetz GmbH,341.0
126,341,DE-NW-007398,Sengenberg,Unbewirtschaftet,A46,2200,POINT (4209717.23309444 3141162.8961384213),Westnetz GmbH,Westnetz GmbH,340.0
127,342,DE-BY-008680,Fürthholz-Süd,Unbewirtschaftet,A94,3960,POINT (4483468.525079381 2796239.8707123846),Stadtwerke Dorfen GmbH,Bayernwerk Netz GmbH,NaN
128,348,DE-BY-000386,Aster Moos Ost,Unbewirtschaftet,A9,3520,POINT (4439424.787947541 2809613.968272738),Bayernwerk Netz GmbH,Bayernwerk Netz GmbH,NaN


In [45]:
import overpy
import json
import time

# Inicializar Overpass API
api = overpy.Overpass()

# Definir Bundesländer und ihre Kürzel für die Abfragen
bundeslaender = {
    "BY": "Bayern",
    "BW": "Baden-Württemberg",
    "HE": "Hessen",
    "NW": "Nordrhein-Westfalen",
    "SL": "Saarland",
    "NI": "Niedersachsen",
    "RP": "Rheinland-Pfalz",
    "BB": "Brandenburg",
    "BE": "Berlin",
    "SH": "Schleswig-Holstein",
    "ST": "Sachsen-Anhalt",
    "SN": "Sachsen",
    "TH": "Thüringen",
    "MV": "Mecklenburg-Vorpommern",
    "HB": "Bremen",
    "HH": "Hamburg"
}

# Liste der Autobahnen mit Leerzeichen in der Referenz
autobahnen = ['A 7', 'A 43', 'A 24', 'A 60', 'A 8', 'A 70', 'A 9', 'A 27', 'A 13', 'A 6', 'A 66', 'A 31', 'A 29', 'A 64', 'A 3', 'A 40', 'A 23', 'A 2', 'A 1', 'A 11', 'A 81', 'A 49', 'A 63', 'A 14', 'A 4', 'A 44', 'A 5', 'A 92', 'A 19', 'A 72', 'A 21', 'A 15', 'A 45', 'A 39', 'A 96', 'A 10', 'A 93', 'A 95', 'A 20', 'A 38', 'A 71', 'A 17', 'A 73', 'A 36', 'A 46', 'A 94']

# Schleife über jedes Bundesland und jede Autobahn
for bundesland_kurz, suchgebiet in bundeslaender.items():
    for autobahn in autobahnen:
        print(f"Verarbeite {autobahn} im Bundesland {suchgebiet}...")

        # Erstellen der Overpass-Abfrage für das spezifische Bundesland und die Autobahn
        overpass_query = f"""
        [out:json][timeout:250];
        area["name"="{suchgebiet}"]["type"="boundary"]->.a;
        (
          way["highway"="motorway"]["ref"="{autobahn}"](area.a);
        );
        (._;>;);
        out meta;
        """

        try:
            # Anfrage an die Overpass API senden
            result = api.query(overpass_query)

            # Überprüfen, ob die Abfrage Daten zurückgibt
            if not result.ways:
                print(f"Keine Daten für {autobahn} im Bundesland {suchgebiet} gefunden.")
                continue

            # Erstellen der Feature-Liste für die GeoJSON-Datei
            features = []
            for way in result.ways:
                coords = [(float(node.lon), float(node.lat)) for node in way.nodes]  # Konvertiere Decimal in float
                feature = {
                    "type": "Feature",
                    "properties": {
                        "id": way.id,
                        "tags": way.tags
                    },
                    "geometry": {
                        "type": "LineString",
                        "coordinates": coords
                    }
                }
                features.append(feature)

            # GeoJSON-Objekt erstellen
            geojson = {
                "type": "FeatureCollection",
                "features": features
            }

            # Speichern der GeoJSON-Datei im gewünschten Verzeichnis
            filename = f"U:\\Desktop\\kmStand\\shapes\\{bundesland_kurz}_{autobahn.replace(' ', '')}.geojson"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(geojson, f, ensure_ascii=False, indent=2)

            print(f"{filename} wurde erfolgreich gespeichert.")
        
        except Exception as e:
            print(f"Fehler beim Abrufen der Daten für {autobahn} im Bundesland {suchgebiet}: {e}")
        
        # Wartezeit zwischen Anfragen, um die API zu entlasten
        time.sleep(1)

print("Alle Autobahnen und Bundesländer wurden verarbeitet.")


Verarbeite A 7 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\BY_A7.geojson wurde erfolgreich gespeichert.
Verarbeite A 43 im Bundesland Bayern...
Keine Daten für A 43 im Bundesland Bayern gefunden.
Verarbeite A 24 im Bundesland Bayern...
Keine Daten für A 24 im Bundesland Bayern gefunden.
Verarbeite A 60 im Bundesland Bayern...
Keine Daten für A 60 im Bundesland Bayern gefunden.
Verarbeite A 8 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\BY_A8.geojson wurde erfolgreich gespeichert.
Verarbeite A 70 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\BY_A70.geojson wurde erfolgreich gespeichert.
Verarbeite A 9 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\BY_A9.geojson wurde erfolgreich gespeichert.
Verarbeite A 27 im Bundesland Bayern...
Keine Daten für A 27 im Bundesland Bayern gefunden.
Verarbeite A 13 im Bundesland Bayern...
Keine Daten für A 13 im Bundesland Bayern gefunden.
Verarbeite A 6 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\BY_A6.geojson wurde erfolgreich g

KeyboardInterrupt: 

In [47]:
import overpy
import json
import pandas as pd
import time

# Cargar el archivo CSV y definir la API de Overpass
df = pd.read_csv('U:\\Desktop\\kmStand\\csv-daten\\rastatteUnbewirtschaft.csv')
api = overpy.Overpass()

# Mapeo de Bundesland-Kürzel a den vollständigen Namen
bundesland_map = {
    "BY": "Bayern",
    "BW": "Baden-Württemberg",
    "HE": "Hessen",
    "NW": "Nordrhein-Westfalen",
    "SL": "Saarland",
    "NI": "Niedersachsen",
    "RP": "Rheinland-Pfalz",
    "BB": "Brandenburg",
    "BE": "Berlin",
    "SH": "Schleswig-Holstein",
    "ST": "Sachsen-Anhalt",
    "SN": "Sachsen",
    "TH": "Thüringen",
    "MV": "Mecklenburg-Vorpommern",
    "HB": "Bremen",
    "HH": "Hamburg"
}

# Filtrar und Sammeln der Kombinationen von Bundesland und Autobahn
combinations = df[['Kennung', 'Bundes-autobahn']].dropna().drop_duplicates()
combinations['Bundesland'] = combinations['Kennung'].str.split('-').str[1]
combinations['Bundesland'] = combinations['Bundesland'].map(bundesland_map)

for _, row in combinations.iterrows():
    bundesland = row['Bundesland']
    autobahn = row['Bundes-autobahn']
    
    # Insertar un espacio entre la letra y el número de la Autobahn
    autobahn = " ".join([autobahn[:1], autobahn[1:]])

    print(f"Verarbeite {autobahn} im Bundesland {bundesland}...")

    # Erstellen der Overpass-Abfrage für das spezifische Bundesland und die Autobahn
    overpass_query = f"""
    [out:json][timeout:250];
    area["name"="{bundesland}"]["type"="boundary"]->.a;
    (
      way["highway"="motorway"]["ref"="{autobahn}"](area.a);
    );
    (._;>;);
    out meta;
    """

    try:
        # Anfrage an die Overpass API senden
        result = api.query(overpass_query)

        # Überprüfen, ob die Abfrage Daten zurückgibt
        if not result.ways:
            print(f"Keine Daten für {autobahn} im Bundesland {bundesland} gefunden.")
            continue

        # Erstellen der Feature-Liste für die GeoJSON-Datei
        features = []
        for way in result.ways:
            coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
            feature = {
                "type": "Feature",
                "properties": {
                    "id": way.id,
                    "tags": way.tags
                },
                "geometry": {
                    "type": "LineString",
                    "coordinates": coords
                }
            }
            features.append(feature)

        # GeoJSON-Objekt erstellen
        geojson = {
            "type": "FeatureCollection",
            "features": features
        }

        # Speichern der GeoJSON-Datei im gewünschten Verzeichnis
        filename = f"U:\\Desktop\\kmStand\\shapes\\{bundesland}_{autobahn}.geojson"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(geojson, f, ensure_ascii=False, indent=2)

        print(f"{filename} wurde erfolgreich gespeichert.")

    except Exception as e:
        print(f"Fehler beim Abrufen der Daten für {autobahn} im Bundesland {bundesland}: {e}")

    # Wartezeit zwischen Anfragen, um die API zu entlasten
    time.sleep(1)

print("Alle Kombinationen von Bundesländern und Autobahnen wurden verarbeitet.")


Verarbeite A 7 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\Bayern_A 7.geojson wurde erfolgreich gespeichert.
Verarbeite A 43 im Bundesland Nordrhein-Westfalen...
U:\Desktop\kmStand\shapes\Nordrhein-Westfalen_A 43.geojson wurde erfolgreich gespeichert.
Verarbeite A 24 im Bundesland Brandenburg...
U:\Desktop\kmStand\shapes\Brandenburg_A 24.geojson wurde erfolgreich gespeichert.
Verarbeite A 60 im Bundesland Rheinland-Pfalz...
U:\Desktop\kmStand\shapes\Rheinland-Pfalz_A 60.geojson wurde erfolgreich gespeichert.
Verarbeite A 8 im Bundesland Saarland...
U:\Desktop\kmStand\shapes\Saarland_A 8.geojson wurde erfolgreich gespeichert.
Verarbeite A 7 im Bundesland Schleswig-Holstein...
U:\Desktop\kmStand\shapes\Schleswig-Holstein_A 7.geojson wurde erfolgreich gespeichert.
Verarbeite A 70 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\Bayern_A 70.geojson wurde erfolgreich gespeichert.
Verarbeite A 9 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\Bayern_A 9.geojson wurde erfolgreich ges

In [52]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

def verarbeite_lkws_auf_autobahn(waypoints_dir, output_dir, shapes_dir):
    # Liste der Tage
    tage = ['04_01']  # Kann erweitert werden
    chunk_size = 20000  # Größe der Chunk-Verarbeitung

    # Iteriere über jede GeoJSON-Datei im shapes-Verzeichnis
    for shape_file in os.listdir(shapes_dir):
        if shape_file.endswith('.geojson'):
            # Autobahnname aus dem Dateinamen extrahieren (z.B., A7.geojson -> A7)
            autobahn_name = shape_file.split('.')[0]
            autobahn_pfad = os.path.join(shapes_dir, shape_file)
            
            # Lesen der GeoJSON-Datei für die jeweilige Autobahn
            autobahn_gdf = gpd.read_file(autobahn_pfad).to_crs('EPSG:25832')

            # Hinzufügen eines Puffers von 50 Metern für Ungenauigkeiten
            autobahn_gdf['geometry'] = autobahn_gdf.geometry.buffer(40)

            print(f"Verarbeite {autobahn_name}...")

            for tag in tage:
                input_csv = os.path.join(waypoints_dir, f'{tag}_DEU_waypoints.csv')
                output_csv = os.path.join(output_dir, f'LKWauf{autobahn_name}_{tag}.csv')
                header_geschrieben = False
                gefundene_trip_ids = set()

                # Waypoints-Datei in Chunks lesen
                for chunk_number, chunk in enumerate(pd.read_csv(input_csv, chunksize=chunk_size)):
                    print(f"Verarbeite Chunk {chunk_number + 1} für Tag {tag} auf {autobahn_name}")
                    
                    # Spalten umbenennen
                    chunk.rename(columns={
                        chunk.columns[0]: "Trip-Id",
                        chunk.columns[1]: "WaypointSequence",
                        chunk.columns[2]: "CaptureDate",
                        chunk.columns[3]: "Latitude",
                        chunk.columns[4]: "Longitude",
                        chunk.columns[5]: "SegmentID",
                        chunk.columns[6]: "ZoneName",
                        chunk.columns[7]: "FRC",
                        chunk.columns[8]: "DeviceId",
                        chunk.columns[9]: "RawSpeed",
                        chunk.columns[10]: "RawSpeedMetric",
                        chunk.columns[11]: "CustomAttrs"
                    }, inplace=True)
                    
                    # Geometrie erstellen und projizieren
                    geometry = [Point(xy) for xy in zip(chunk['Longitude'], chunk['Latitude'])]
                    chunk_gdf = gpd.GeoDataFrame(chunk, geometry=geometry, crs='EPSG:4326').to_crs('EPSG:25832')
                    
                    # Räumliche Vereinigung mit der Autobahn (mit Puffer von 50 Metern)
                    joined = gpd.sjoin(chunk_gdf, autobahn_gdf, how='inner', predicate='intersects')
                    
                    if not joined.empty:
                        print(f"LKWs auf {autobahn_name} im Chunk {chunk_number + 1} für den Tag {tag}")
                        
                        # Neue Trip-Ids filtern
                        neue_trip_ids = set(joined['Trip-Id']) - gefundene_trip_ids
                        if neue_trip_ids:
                            gefundene_trip_ids.update(neue_trip_ids)
                            gefilterter_chunk = chunk[chunk['Trip-Id'].isin(neue_trip_ids)]
                            
                            # In CSV-Datei schreiben
                            if not header_geschrieben:
                                gefilterter_chunk.to_csv(output_csv, index=False, mode='w')
                                header_geschrieben = True
                            else:
                                gefilterter_chunk.to_csv(output_csv, index=False, header=False, mode='a')
                        else:
                            print("Alle Trip-Ids aus diesem Chunk wurden bereits verarbeitet.")
                    else:
                        print(f"Keine LKWs auf {autobahn_name} für den Tag {tag} in diesem Chunk gefunden.")
                
                print(f"Verarbeitung für den Tag {tag} auf {autobahn_name} abgeschlossen. Ergebnisse gespeichert in {output_csv}")

    print("Alle Daten wurden verarbeitet.")

# Verzeichnisse
waypoints_verzeichnis = r'Z:\Lars\TENT\DEU\year=2019\waypoints'
output_verzeichnis = r'U:\Desktop\kmStand\csv-daten'
shapes_verzeichnis = r'U:\Desktop\kmStand\shapes'

# Funktion aufrufen
verarbeite_lkws_auf_autobahn(waypoints_verzeichnis, output_verzeichnis, shapes_verzeichnis)


Verarbeite Baden-Württemberg_A 5...
Verarbeite Chunk 1 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 1 für den Tag 04_01
Verarbeite Chunk 2 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 2 für den Tag 04_01
Verarbeite Chunk 3 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 3 für den Tag 04_01
Verarbeite Chunk 4 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 4 für den Tag 04_01
Verarbeite Chunk 5 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 5 für den Tag 04_01
Verarbeite Chunk 6 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 6 für den Tag 04_01
Verarbeite Chunk 7 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 7 für den Tag 04_01
Verarbeite Chunk 8 für Tag 04_01 auf Baden-Württemberg_A 5
LKWs auf Baden-Württemberg_A 5 im Chunk 8 für den Tag 04_01
Verarbeite C

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [54]:
df = pd.read_csv("U:\Desktop\kmStand\csv-daten\LKWaufBaden-Württemberg_A 5_04_01.csv")
df

,Trip-Id,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentID,ZoneName,FRC,DeviceId,RawSpeed,RawSpeedMetric,CustomAttrs
0,ff18c7c95e0ccedd07d4a3cc50322260,0,2019-04-01T12:00:22.022Z,49.003090,8.329520,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,14.0,kph,NaN
1,ff18c7c95e0ccedd07d4a3cc50322260,1,2019-04-01T12:00:55.055Z,49.001565,8.331662,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,27.0,kph,NaN
2,ff18c7c95e0ccedd07d4a3cc50322260,2,2019-04-01T12:00:59.059Z,49.001323,8.331863,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,24.0,kph,NaN
3,ff18c7c95e0ccedd07d4a3cc50322260,3,2019-04-01T12:01:02.002Z,49.001183,8.332007,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,18.0,kph,NaN
4,ff18c7c95e0ccedd07d4a3cc50322260,4,2019-04-01T12:01:04.004Z,49.001162,8.332167,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,21.0,kph,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
635242,01ffab4d1c3937109986bd39deb016b4,277,2019-04-01T13:35:43.043Z,47.212737,5.841532,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,87.0,kph,NaN
635243,01ffab4d1c3937109986bd39deb016b4,278,2019-04-01T13:37:08.008Z,47.199853,5.822688,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,77.0,kph,NaN
635244,01ffab4d1c3937109986bd39deb016b4,279,2019-04-01T13:38:39.039Z,47.193825,5.797533,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,89.0,kph,NaN
635245,01ffab4d1c3937109986bd39deb016b4,280,2019-04-01T13:40:02.002Z,47.193795,5.770803,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,87.0,kph,NaN


In [55]:
import pandas as pd

# Laden Sie die Daten aus der CSV-Datei
# Extrahieren Sie die eindeutigen Trip-Ids
unique_trip_ids = df['Trip-Id'].unique()

# Zählen Sie die Anzahl der eindeutigen Trip-Ids
anzahl_autos = len(unique_trip_ids)

print(f"Die Anzahl der verschiedenen Autos (Trip-Ids) in 'waypoints_towards_ludwigsburg.csv' beträgt: {anzahl_autos}")


Die Anzahl der verschiedenen Autos (Trip-Ids) in 'waypoints_towards_ludwigsburg.csv' beträgt: 2787


### TEST

In [19]:
import pandas as pd
import folium
import os

# Load the CSV file
df = pd.read_csv(r"U:\Desktop\kmStand\csv-daten\LKWaufBaden-Württemberg_A 5_04_01.csv")

# Get all unique Trip-Ids
unique_trip_ids = df['Trip-Id'].unique()

# Directory to save maps
output_dir = r"U:\Desktop\kmStand\html-maps"
os.makedirs(output_dir, exist_ok=True)

# Loop over each Trip-Id
for trip_id in unique_trip_ids:
    trip_data = df[df['Trip-Id'] == trip_id]
    
    # Get starting coordinates for map center
    start_lat = trip_data['Latitude'].iloc[0]
    start_lon = trip_data['Longitude'].iloc[0]

    # Create map centered at the starting location
    map_osm = folium.Map(location=[start_lat, start_lon], zoom_start=14)

    # Plot each point for the selected Trip-Id
    for _, row in trip_data.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"Speed: {row['RawSpeed']} {row['RawSpeedMetric']}"
        ).add_to(map_osm)

    # Add the GeoJSON layer
    geojson_path = r"U:\Desktop\kmStand\shapes\Baden-Württemberg_A 5.geojson"
    if os.path.exists(geojson_path):
        folium.GeoJson(
            geojson_path,
            name="GeoJSON Data",
            style_function=lambda x: {'color': 'blue', 'weight': 2}
        ).add_to(map_osm)
    else:
        print(f"GeoJSON file not found at: {geojson_path}")

    # Add layer control
    folium.LayerControl().add_to(map_osm)

    # Save each map as an HTML file, named by Trip-Id
    map_path = os.path.join(output_dir, f"map_trip_{trip_id}.html")
    map_osm.save(map_path)

    print(f"Map saved for Trip-Id {trip_id} at {map_path}")


Map saved for Trip-Id ff18c7c95e0ccedd07d4a3cc50322260 at U:\Desktop\kmStand\html-maps\map_trip_ff18c7c95e0ccedd07d4a3cc50322260.html
Map saved for Trip-Id ff1ac847d0952bb324f095f0551f4a0f at U:\Desktop\kmStand\html-maps\map_trip_ff1ac847d0952bb324f095f0551f4a0f.html
Map saved for Trip-Id ff1b2e46b4011c37fac31bd36dc70d0e at U:\Desktop\kmStand\html-maps\map_trip_ff1b2e46b4011c37fac31bd36dc70d0e.html
Map saved for Trip-Id ff3207b2991a75e832c2cbea75acd133 at U:\Desktop\kmStand\html-maps\map_trip_ff3207b2991a75e832c2cbea75acd133.html
Map saved for Trip-Id ff3e84df00cc0edd4353061cbd1081a7 at U:\Desktop\kmStand\html-maps\map_trip_ff3e84df00cc0edd4353061cbd1081a7.html
Map saved for Trip-Id ff406b72605cc6b2d7854aa430ee7a69 at U:\Desktop\kmStand\html-maps\map_trip_ff406b72605cc6b2d7854aa430ee7a69.html
Map saved for Trip-Id ff416764a2349062d146244c0fe5a92c at U:\Desktop\kmStand\html-maps\map_trip_ff416764a2349062d146244c0fe5a92c.html
Map saved for Trip-Id ff54d596e787dd169c4c2fd597578dd4 at U:\D

KeyboardInterrupt: 

# WICHTIG  

In [97]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium
from folium import Marker
from shapely.ops import linemerge

# Funktion zum Berechnen der Punkte vor und hinter dem benutzerdefinierten Punkt
def berechne_punkte(carretera_gdf, benutzerkoordinaten, abstand=1000):
    # Konvertieren der Geometrie in UTM für metrische Maßeinheiten
    carretera_gdf = carretera_gdf.to_crs(epsg=32632)
    
    # Zusammenfügen der Segmente zu einer durchgehenden Linie
    lineas = list(carretera_gdf.geometry)
    carretera_unida = linemerge(lineas)
    
    # Umwandeln des benutzerdefinierten Punkts in UTM-Koordinaten
    benutzerpunkt_utm = gpd.GeoSeries([Point(benutzerkoordinaten)], crs="EPSG:4326").to_crs(epsg=32632).iloc[0]
    
    # Den nächsten Punkt auf der Linie finden
    startpunkt = carretera_unida.project(benutzerpunkt_utm)
    
    # Berechnen der Punkte 20 km davor und danach
    punkt_vorher = carretera_unida.interpolate(startpunkt - abstand)
    punkt_nachher = carretera_unida.interpolate(startpunkt + abstand)
    
    # Zurückprojizieren der Punkte auf WGS84 für OpenStreetMap
    carretera_gdf = carretera_gdf.to_crs("EPSG:4326")
    benutzerpunkt = gpd.GeoSeries([benutzerpunkt_utm], crs="EPSG:32632").to_crs("EPSG:4326").iloc[0]
    punkt_vorher = gpd.GeoSeries([punkt_vorher], crs="EPSG:32632").to_crs("EPSG:4326").iloc[0]
    punkt_nachher = gpd.GeoSeries([punkt_nachher], crs="EPSG:32632").to_crs("EPSG:4326").iloc[0]
    
    return benutzerpunkt, punkt_vorher, punkt_nachher

# Laden der GeoJSON-Datei der Autobahn
carretera_gdf = gpd.read_file(r"U:\Desktop\kmStand\shapes\Baden-Württemberg_A 5.geojson")

# Definieren der benutzerdefinierten Koordinaten (Längengrad, Breitengrad)
benutzerkoordinaten = (8.63706988606853, 49.406358828201455)  # Beispielkoordinaten
#49.405071879041934, 8.63706988606853
#daneben : 
#49.406358828201455, 8.638308887138146

# Berechnen der benachbarten Punkte
benutzerpunkt, punkt_vorher, punkt_nachher = berechne_punkte(carretera_gdf, benutzerkoordinaten)

# Koordinaten extrahieren
coords_benutzerpunkt = (benutzerpunkt.y, benutzerpunkt.x)
coords_punkt_vorher = (punkt_vorher.y, punkt_vorher.x)
coords_punkt_nachher = (punkt_nachher.y, punkt_nachher.x)

# Erstellen der Karte in OpenStreetMap
mapa = folium.Map(location=coords_benutzerpunkt, zoom_start=10)

# Hinzufügen der Straße zur Karte
for line in carretera_gdf.geometry:
    folium.GeoJson(line).add_to(mapa)

# Hinzufügen der Punkte zur Karte
Marker(coords_benutzerpunkt, tooltip="Benutzerdefinierter Punkt").add_to(mapa)
Marker(coords_punkt_vorher, tooltip="Punkt 20 km vorher").add_to(mapa)
Marker(coords_punkt_nachher, tooltip="Punkt 20 km nachher").add_to(mapa)

# Karte speichern und anzeigen
mapa.save("strasse_mit_benutzerdefiniertem_punkt.html")
mapa


In [98]:
df = pd.read_csv('U:\\Desktop\\kmStand\\csv-daten\\LKWaufBaden-Württemberg_A 5_04_01.csv')
df

,Trip-Id,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentID,ZoneName,FRC,DeviceId,RawSpeed,RawSpeedMetric,CustomAttrs
0,ff18c7c95e0ccedd07d4a3cc50322260,0,2019-04-01T12:00:22.022Z,49.003090,8.329520,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,14.0,kph,NaN
1,ff18c7c95e0ccedd07d4a3cc50322260,1,2019-04-01T12:00:55.055Z,49.001565,8.331662,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,27.0,kph,NaN
2,ff18c7c95e0ccedd07d4a3cc50322260,2,2019-04-01T12:00:59.059Z,49.001323,8.331863,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,24.0,kph,NaN
3,ff18c7c95e0ccedd07d4a3cc50322260,3,2019-04-01T12:01:02.002Z,49.001183,8.332007,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,18.0,kph,NaN
4,ff18c7c95e0ccedd07d4a3cc50322260,4,2019-04-01T12:01:04.004Z,49.001162,8.332167,NaN,NaN,NaN,4ce17dd0371a2844b225f74f39de8609,21.0,kph,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
635242,01ffab4d1c3937109986bd39deb016b4,277,2019-04-01T13:35:43.043Z,47.212737,5.841532,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,87.0,kph,NaN
635243,01ffab4d1c3937109986bd39deb016b4,278,2019-04-01T13:37:08.008Z,47.199853,5.822688,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,77.0,kph,NaN
635244,01ffab4d1c3937109986bd39deb016b4,279,2019-04-01T13:38:39.039Z,47.193825,5.797533,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,89.0,kph,NaN
635245,01ffab4d1c3937109986bd39deb016b4,280,2019-04-01T13:40:02.002Z,47.193795,5.770803,NaN,NaN,NaN,324fb91e720bcee969eddd8e7037f505,87.0,kph,NaN


# Plan: 

-LKWs Inrix auf Autobahn filtrieren

- First: Get from excel the Autobahn Number and with Kennung the Bundesland and coordinates. 

- With Autobahn Nummer and Bundesland the geojson 

- With coordinates, two points before and after, save before and after coordinates

- with before and after coordinates, Inrix if condition

- The LKWs that drive through, in one csv sheet

- count the LKWs that drive through

## functions 

### Extraction of Info. 

In [15]:
import pandas as pd

def UnbewirtschaftAutobahnArray():
    # Cargar el archivo Excel y establecer el encabezado en la segunda fila
    df = pd.read_excel('U:\\Desktop\\kmStand\\csv-daten\\merged_Lkw_Ladenetz_Stellplatzdaten.xlsx')


    
    # Filtrar las filas donde la columna 'Bewirtschaftung' es 'Unbewirtschaftet'
    unbewirtschaft_df = df[df['Bewirtschaftung_locationData'] == 'Unbewirtschaftet']
    
    # Extraer las Autobahnen 'Unbewirtschaftet' en una lista única
    unbewirtschaftete_autobahnen = unbewirtschaft_df['Bundes_autobahn'].unique().tolist()
    
    # Guardar la información completa de 'Unbewirtschaftet' en un archivo CSV
    unbewirtschaft_df.to_csv('U:\\Desktop\\kmStand\\csv-daten\\rastatteUnbewirtschaft.csv', index=False)
    
    # Mostrar el resultado
    print("Unbewirtschaftete Autobahnen:", unbewirtschaftete_autobahnen)

    return unbewirtschaftete_autobahnen

# Llamada a la función
unbewirtschaftete_autobahnen = UnbewirtschaftAutobahnArray()


Unbewirtschaftete Autobahnen: ['A10', 'A9', 'A3', 'A8', 'A6', 'A2', 'A40', 'A92', 'A60', 'A70', 'A20', 'A44', 'A46', 'A5', 'A7', 'A64', 'A39', 'A11', 'A95', 'A24', 'A14', 'A1', 'A71', 'A31', 'A72', 'A81', 'A19', 'A43', 'A23', 'A66', 'A27', 'A38', 'A94', 'A96', 'A13', 'A15', 'A45', 'A63', 'A93', 'A21', 'A73', 'A4', 'A36', 'A29', 'A49', 'A17']


In [29]:
import pandas as pd
import geopandas as gpd
from pyproj import Transformer

def bundesland_und_koordinaten_ermitteln(csv_verzeichnis):
    # CSV-Datei einlesen
    df = pd.read_csv(csv_verzeichnis)
    
    # Liste zum Speichern der Ergebnisse erstellen
    ergebnisse = []
    
    # Transformator von UTM Zone 32N zu WGS84 (Breite/Länge) für Deutschland
    transformer = Transformer.from_crs("EPSG:25832", "EPSG:4326", always_xy=True)
    
    # Dictionary zur Umwandlung des Bundeslandcodes in den vollständigen Namen
    bundesland_dict = {
        "BW": "Baden-Württemberg",
        "BY": "Bayern",
        "BE": "Berlin",
        "BB": "Brandenburg",
        "HB": "Bremen",
        "HH": "Hamburg",
        "HE": "Hessen",
        "MV": "Mecklenburg-Vorpommern",
        "NI": "Niedersachsen",
        "NW": "Nordrhein-Westfalen",
        "RP": "Rheinland-Pfalz",
        "SL": "Saarland",
        "SN": "Sachsen",
        "ST": "Sachsen-Anhalt",
        "SH": "Schleswig-Holstein",
        "TH": "Thüringen"
    }
    
    # Jede Zeile verarbeiten
    for index, zeile in df.iterrows():
        # Bundesland aus der Spalte "Kennung" ermitteln (die ersten zwei Buchstaben)
        bundesland_code = zeile["Kennung"].split("-")[1]
        bundesland = bundesland_dict.get(bundesland_code, "Unbekannt")
        
        # Autobahnname anpassen, z.B. "A5" zu "A 5"
        autobahn = zeile["Bundes_autobahn"]
        autobahn_format = f"{autobahn[0]} {autobahn[1:]}"  # Fügt ein Leerzeichen zwischen Buchstabe und Nummer ein
        
        # Geografische Länge und Breite direkt verwenden (ohne WKT-Parsing)
        laenge = zeile["Geogr_L_nge"]
        breite = zeile["Geogr_Breite"]
        
        # Ergebnis speichern
        ergebnisse.append({
            "Kennung": zeile["Kennung"],
            "Bundesland": bundesland,
            "Autobahn": autobahn_format,
            "Breite": breite,
            "Länge": laenge
        })
    
    # Ergebnisse in ein DataFrame konvertieren, um sie einfach anzuzeigen
    ergebnisse_df = pd.DataFrame(ergebnisse)
    return ergebnisse_df

# Beispielaufruf
csv_verzeichnis = r"U:\Desktop\kmStand\csv-daten\rastatteUnbewirtschaft.csv"
ergebnisse_df = bundesland_und_koordinaten_ermitteln(csv_verzeichnis)
print(ergebnisse_df)


          Kennung           Bundesland Autobahn     Breite      Länge
0    DE-BB-002402          Brandenburg     A 10  52,704304  13,065977
1    DE-BY-000359               Bayern      A 9  49,548934  11,321682
2    DE-NW-000102  Nordrhein-Westfalen      A 3  51,778662   6,515125
3    DE-NW-000108  Nordrhein-Westfalen      A 3  51,704427   6,640445
4    DE-SL-000520             Saarland      A 8  49,453827   6,568357
..            ...                  ...      ...        ...        ...
125  DE-SN-000040              Sachsen      A 4   51,18797  14,985706
126  DE-SN-000039              Sachsen      A 4  51,185535  14,993305
127  DE-HE-490004               Hessen     A 49  51,192034   9,395979
128  DE-SN-000012              Sachsen     A 17  50,821588   13,88753
129  DE-NW-007399  Nordrhein-Westfalen     A 46  51,366258   8,410971

[130 rows x 5 columns]


### Overpass Api. 

In [21]:
import overpy
import json
import time

# Funktion zur Verarbeitung der Kombinationen von Bundesland und Autobahn
def verarbeite_bundesland_autobahn(ergebnisse_df, api, output_dir):
    for _, row in ergebnisse_df.iterrows():
        kennung = row['Kennung']
        bundesland = row['Bundesland']
        autobahn = row['Autobahn']
        
        print(f"Verarbeite {autobahn} im Bundesland {bundesland}...")

        # Erstellen der Overpass-Abfrage für das spezifische Bundesland und die Autobahn
        overpass_query = f"""
        [out:json][timeout:250];
        area["name"="{bundesland}"]["type"="boundary"]->.a;
        (
          way["highway"="motorway"]["ref"="{autobahn}"](area.a);
        );
        (._;>;);
        out meta;
        """

        try:
            # Anfrage an die Overpass API senden
            result = api.query(overpass_query)

            # Überprüfen, ob die Abfrage Daten zurückgibt
            if not result.ways:
                print(f"Keine Daten für {autobahn} im Bundesland {bundesland} gefunden.")
                continue

            # Erstellen der Feature-Liste für die GeoJSON-Datei
            features = []
            for way in result.ways:
                coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
                feature = {
                    "type": "Feature",
                    "properties": {
                        "id": way.id,
                        "tags": way.tags
                    },
                    "geometry": {
                        "type": "LineString",
                        "coordinates": coords
                    }
                }
                features.append(feature)

            # GeoJSON-Objekt erstellen
            geojson = {
                "type": "FeatureCollection",
                "features": features
            }

            # Speichern der GeoJSON-Datei im gewünschten Verzeichnis
            filename = f"{output_dir}\\{kennung}.geojson"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(geojson, f, ensure_ascii=False, indent=2)

            print(f"{filename} wurde erfolgreich gespeichert.")

        except Exception as e:
            print(f"Fehler beim Abrufen der Daten für {autobahn} im Bundesland {bundesland}: {e}")

        # Wartezeit zwischen Anfragen, um die API zu entlasten
        time.sleep(1)

    print("Alle Kombinationen von Bundesländern und Autobahnen wurden verarbeitet.")

# Beispielaufruf der Funktion
api = overpy.Overpass()
output_dir = r"U:\Desktop\kmStand\shapes"
csv_verzeichnis = r"U:\Desktop\kmStand\csv-daten\rastatteUnbewirtschaft.csv"

# Hier ergebnisse_df verwenden, das bereits erstellt wurde
verarbeite_bundesland_autobahn(ergebnisse_df, api, output_dir)


Verarbeite A 10 im Bundesland Brandenburg...
U:\Desktop\kmStand\shapes\DE-BB-002402.geojson wurde erfolgreich gespeichert.
Verarbeite A 9 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\DE-BY-000359.geojson wurde erfolgreich gespeichert.
Verarbeite A 3 im Bundesland Nordrhein-Westfalen...
U:\Desktop\kmStand\shapes\DE-NW-000102.geojson wurde erfolgreich gespeichert.
Verarbeite A 3 im Bundesland Nordrhein-Westfalen...
U:\Desktop\kmStand\shapes\DE-NW-000108.geojson wurde erfolgreich gespeichert.
Verarbeite A 8 im Bundesland Saarland...
U:\Desktop\kmStand\shapes\DE-SL-000520.geojson wurde erfolgreich gespeichert.
Verarbeite A 6 im Bundesland Baden-Württemberg...
U:\Desktop\kmStand\shapes\DE-BW-006050.geojson wurde erfolgreich gespeichert.
Verarbeite A 2 im Bundesland Nordrhein-Westfalen...
U:\Desktop\kmStand\shapes\DE-NW-000071.geojson wurde erfolgreich gespeichert.
Verarbeite A 6 im Bundesland Bayern...
U:\Desktop\kmStand\shapes\DE-BY-000183.geojson wurde erfolgreich gespeichert.
Verarb

### Punkte berechnen

In [34]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium import Marker
from shapely.ops import linemerge
from pyproj import Transformer
import os

def berechne_punkte_und_erzeuge_karte_dynamisch(ergebnisse_df, shapes_directory, abstand=1000):
    # Transformator von WGS84 zu UTM (EPSG:32632)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32632", always_xy=True)
    
    ergebnisse = []

    # Sicherstellen, dass Länge und Breite das richtige Dezimalformat haben
    ergebnisse_df['Länge'] = ergebnisse_df['Länge'].astype(str).str.replace(",", ".").astype(float)
    ergebnisse_df['Breite'] = ergebnisse_df['Breite'].astype(str).str.replace(",", ".").astype(float)

    for index, row in ergebnisse_df.iterrows():
        # Laden der spezifischen GeoJSON-Datei basierend auf der Kennung
        kennung = row['Kennung']
        geojson_file = os.path.join(shapes_directory, f"{kennung}.geojson")
        
        if not os.path.exists(geojson_file):
            print(f"GeoJSON-Datei für {kennung} nicht gefunden, überspringe...")
            continue

        carretera_gdf = gpd.read_file(geojson_file)
        
        # Konvertieren der Geometrie der Autobahn in UTM für metrische Maßeinheiten
        carretera_gdf = carretera_gdf.to_crs(epsg=32632)
        lineas = list(carretera_gdf.geometry)
        carretera_unida = linemerge(lineas)
        
        # Umwandeln des benutzerdefinierten Punkts in UTM-Koordinaten
        lon, lat = row['Länge'], row['Breite']
        x, y = transformer.transform(lon, lat)
        benutzerpunkt_utm = Point(x, y)

        # Den nächsten Punkt auf der Linie finden
        startpunkt = carretera_unida.project(benutzerpunkt_utm)
        
        # Berechnen der Punkte vor und nach dem Punkt
        punkt_vorher = carretera_unida.interpolate(max(startpunkt - abstand, 0))
        punkt_nachher = carretera_unida.interpolate(min(startpunkt + abstand, carretera_unida.length))
        
        # Zurückprojizieren der Punkte auf WGS84 für OpenStreetMap
        transformer_to_wgs = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True)
        benutzerpunkt_wgs = Point(*transformer_to_wgs.transform(benutzerpunkt_utm.x, benutzerpunkt_utm.y))
        punkt_vorher_wgs = Point(*transformer_to_wgs.transform(punkt_vorher.x, punkt_vorher.y))
        punkt_nachher_wgs = Point(*transformer_to_wgs.transform(punkt_nachher.x, punkt_nachher.y))
        
        # Speichern der Ergebnisse
        ergebnisse.append({
            "Kennung": row["Kennung"],
            "Bundesland": row["Bundesland"],
            "Koordinate_Benutzer": (benutzerpunkt_wgs.y, benutzerpunkt_wgs.x),
            "Koordinate_Punkt_vorher": (punkt_vorher_wgs.y, punkt_vorher_wgs.x),
            "Koordinate_Punkt_nachher": (punkt_nachher_wgs.y, punkt_nachher_wgs.x)
        })
        
        # Erstellen der Karte in OpenStreetMap
        mapa = folium.Map(location=(benutzerpunkt_wgs.y, benutzerpunkt_wgs.x), zoom_start=10)
        
        # Hinzufügen der Straße zur Karte
        for line in carretera_gdf.to_crs("EPSG:4326").geometry:
            folium.GeoJson(line).add_to(mapa)
        
        # Hinzufügen der Punkte zur Karte
        Marker((benutzerpunkt_wgs.y, benutzerpunkt_wgs.x), tooltip="Benutzerdefinierter Punkt").add_to(mapa)
        Marker((punkt_vorher_wgs.y, punkt_vorher_wgs.x), tooltip="Punkt vor dem Benutzerdefinierten Punkt").add_to(mapa)
        Marker((punkt_nachher_wgs.y, punkt_nachher_wgs.x), tooltip="Punkt nach dem Benutzerdefinierten Punkt").add_to(mapa)
        
        # Karte speichern
        output_directory = "U:\\Desktop\\kmStand\\maps"
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
            
        filename = os.path.join(output_directory, f"{kennung}.html")
        mapa.save(filename)
        print(f"Karte für {kennung} gespeichert unter: {filename}")

    return pd.DataFrame(ergebnisse)

# Beispielaufruf der Funktion
shapes_directory = r"U:\Desktop\kmStand\shapes"  # Verzeichnis der GeoJSON-Dateien
berechnete_ergebnisse_df = berechne_punkte_und_erzeuge_karte_dynamisch(ergebnisse_df, shapes_directory)

# Ausgabe der Ergebnisse
print(berechnete_ergebnisse_df)


Karte für DE-BB-002402 gespeichert unter: U:\Desktop\kmStand\maps\DE-BB-002402.html
Karte für DE-BY-000359 gespeichert unter: U:\Desktop\kmStand\maps\DE-BY-000359.html
Karte für DE-NW-000102 gespeichert unter: U:\Desktop\kmStand\maps\DE-NW-000102.html
Karte für DE-NW-000108 gespeichert unter: U:\Desktop\kmStand\maps\DE-NW-000108.html
Karte für DE-SL-000520 gespeichert unter: U:\Desktop\kmStand\maps\DE-SL-000520.html
Karte für DE-BW-006050 gespeichert unter: U:\Desktop\kmStand\maps\DE-BW-006050.html
Karte für DE-NW-000071 gespeichert unter: U:\Desktop\kmStand\maps\DE-NW-000071.html
Karte für DE-BY-000183 gespeichert unter: U:\Desktop\kmStand\maps\DE-BY-000183.html
Karte für DE-BY-000182 gespeichert unter: U:\Desktop\kmStand\maps\DE-BY-000182.html
Karte für DE-NW-000176 gespeichert unter: U:\Desktop\kmStand\maps\DE-NW-000176.html
Karte für DE-BY-000401 gespeichert unter: U:\Desktop\kmStand\maps\DE-BY-000401.html
Karte für DE-BY-000402 gespeichert unter: U:\Desktop\kmStand\maps\DE-BY-0004

In [35]:
berechnete_ergebnisse_df

,Kennung,Bundesland,Koordinate_Benutzer,Koordinate_Punkt_vorher,Koordinate_Punkt_nachher
0,DE-BB-002402,Brandenburg,"(52.704304, 13.065977)","(52.706201428266446, 13.051478217251745)","(52.70471352116431, 13.080784964032688)"
1,DE-BY-000359,Bayern,"(49.548933999999996, 11.321681999999997)","(49.55775097729742, 11.324390198213514)","(49.540121710053334, 11.318936574727662)"
2,DE-NW-000102,Nordrhein-Westfalen,"(51.77866199999999, 6.515125000000001)","(51.78364445898117, 6.503197248709259)","(51.774627095387956, 6.528067002445739)"
3,DE-NW-000108,Nordrhein-Westfalen,"(51.704426999999995, 6.640445000000001)","(51.70886965578528, 6.628002149365278)","(51.702222525024986, 6.65446800051259)"
4,DE-SL-000520,Saarland,"(49.453827, 6.568357)","(49.453763660321094, 6.554760344677543)","(49.45822144911123, 6.580270501208867)"
...,...,...,...,...,...
125,DE-SN-000040,Sachsen,"(51.18797, 14.985706000000002)","(51.19303826500168, 14.97392311271037)","(51.18412640904006, 14.9985787600799)"
126,DE-SN-000039,Sachsen,"(51.18553500000001, 14.993305)","(51.18999857451222, 14.980971190256367)","(51.181681288669736, 15.00618304910894)"
127,DE-HE-490004,Hessen,"(51.19203399999999, 9.395979)","(51.18723789351512, 9.384334040591844)","(51.19982841575508, 9.402707916542582)"
128,DE-SN-000012,Sachsen,"(50.82158799999999, 13.88753)","(50.83047217465689, 13.886230992508315)","(50.81302902789885, 13.883439896646845)"


### inrix

In [46]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import tqdm  # Para la barra de progreso

def inrix_mit_zählstelle(waypoints_directory, output_directory, berechnete_ergebnisse_df, dias=['04_01']):
    chunk_size = 10000
    puffer_meters = 100  # Definieren des Pufferradius in Metern

    for dia in dias:
        input_csv = os.path.join(waypoints_directory, f'{dia}_DEU_waypoints.csv')
        
        # Iterieren durch berechnete_ergebnisse_df für jede Zählstelle
        for index, row in tqdm.tqdm(berechnete_ergebnisse_df.iterrows(), total=berechnete_ergebnisse_df.shape[0], desc=f"Processing day {dia}", unit="Zählstelle"):
            kennung = row["Kennung"]
            output_csv = os.path.join(output_directory, f'{kennung}_{dia}.csv')
            
            # Überprüfen, ob die Datei bereits existiert
            if os.path.exists(output_csv):
                print(f"Datei für Kennung {kennung} und Tag {dia} existiert bereits. Überspringe...")
                continue
            
            header_written = False
            found_trip_ids = set()
            
            print(f"Verarbeite Zählstelle mit Kennung {kennung} für den Tag {dia}")

            # Iterieren durch die INRIX-Waypoints-Daten
            for chunk_number, chunk in enumerate(pd.read_csv(input_csv, chunksize=chunk_size)):
                print(f"Verarbeite Chunk {chunk_number + 1} für den Tag {dia}")
                
                # Spalten umbenennen für bessere Lesbarkeit
                chunk.rename(columns={
                    chunk.columns[0]: "Trip-Id",
                    chunk.columns[1]: "WaypointSequence",
                    chunk.columns[2]: "CaptureDate",
                    chunk.columns[3]: "Latitude",
                    chunk.columns[4]: "Longitude",
                    chunk.columns[5]: "SegmentID",
                    chunk.columns[6]: "ZoneName",
                    chunk.columns[7]: "FRC",
                    chunk.columns[8]: "DeviceId",
                    chunk.columns[9]: "RawSpeed",
                    chunk.columns[10]: "RawSpeedMetric",
                    chunk.columns[11]: "CustomAttrs"
                }, inplace=True)

                # Erstellen der Geometrie
                geometry = [Point(xy) for xy in zip(chunk['Longitude'], chunk['Latitude'])]
                chunk_gdf = gpd.GeoDataFrame(chunk, geometry=geometry, crs='EPSG:4326').to_crs('EPSG:25832')

                # Koordinaten für die Zählstelle und die Punkte davor und danach
                punkt_vorher = Point(row["Koordinate_Punkt_vorher"][1], row["Koordinate_Punkt_vorher"][0])
                zählstelle_punkt = Point(row["Koordinate_Benutzer"][1], row["Koordinate_Benutzer"][0])
                punkt_nachher = Point(row["Koordinate_Punkt_nachher"][1], row["Koordinate_Punkt_nachher"][0])

                # Erstellen von Puffern um die Punkte
                punkt_gdf = gpd.GeoDataFrame(
                    {'geometry': [punkt_vorher, zählstelle_punkt, punkt_nachher]},
                    crs='EPSG:4326'
                ).to_crs('EPSG:25832')
                
                punkt_gdf['geometry'] = punkt_gdf.geometry.buffer(puffer_meters)

                # Verbinden der Waypoints mit den gepufferten Zählstellen-Punkten
                joined = gpd.sjoin(chunk_gdf, punkt_gdf, how='inner', predicate='intersects')

                # Bedingung für das Speichern basierend auf der Position (vorher, zählstelle, nachher)
                if not joined.empty:
                    print(f"Gefundene LKWs bei Zählstelle in Chunk {chunk_number + 1} für Tag {dia}")

                    # Extrahieren der eindeutigen Trip-IDs
                    new_trip_ids = set(joined['Trip-Id']) - found_trip_ids
                    if new_trip_ids:
                        found_trip_ids.update(new_trip_ids)

                        filtered_chunk = chunk[chunk['Trip-Id'].isin(new_trip_ids)]

                        # Speichern der Daten in das CSV-Datei
                        filtered_chunk.to_csv(output_csv, index=False, mode='a' if header_written else 'w', header=not header_written)
                        header_written = True

            print(f"Verarbeitung für Zählstelle {kennung} und Tag {dia} abgeschlossen. Ergebnisse in {output_csv} gespeichert.")

    print("Alle Zählstellen wurden verarbeitet.")

# Beispielverzeichnisse und Aufruf der Funktion
output_directory = r"U:\Desktop\kmStand\csv-daten"
waypoints_directory = r"Z:\Lars\TENT\DEU\year=2019\waypoints"
# Beispielaufruf
inrix_mit_zählstelle(waypoints_directory, output_directory, berechnete_ergebnisse_df, dias=['04_01'])


Processing day 04_01:   2%|▊                                                   | 2/130 [00:00<00:13,  9.19Zählstelle/s]

Datei für Kennung DE-BB-002402 und Tag 04_01 existiert bereits. Überspringe...
Datei für Kennung DE-BY-000359 und Tag 04_01 existiert bereits. Überspringe...


Processing day 04_01:   3%|█▌                                                  | 4/130 [00:00<00:15,  8.29Zählstelle/s]

Datei für Kennung DE-NW-000102 und Tag 04_01 existiert bereits. Überspringe...
Datei für Kennung DE-NW-000108 und Tag 04_01 existiert bereits. Überspringe...


Processing day 04_01:   5%|██▍                                                 | 6/130 [00:00<00:14,  8.46Zählstelle/s]

...

Datei für Kennung DE-BY-000386 und Tag 04_01 existiert bereits. Überspringe...
Datei für Kennung DE-SN-000040 und Tag 04_01 existiert bereits. Überspringe...


Processing day 04_01:  98%|█████████████████████████████████████████████████▏| 128/130 [00:15<00:00,  7.95Zählstelle/s]

Datei für Kennung DE-SN-000039 und Tag 04_01 existiert bereits. Überspringe...
Datei für Kennung DE-HE-490004 und Tag 04_01 existiert bereits. Überspringe...


Processing day 04_01: 100%|██████████████████████████████████████████████████| 130/130 [00:15<00:00,  8.25Zählstelle/s]

Datei für Kennung DE-SN-000012 und Tag 04_01 existiert bereits. Überspringe...
Datei für Kennung DE-NW-007399 und Tag 04_01 existiert bereits. Überspringe...
Alle Zählstellen wurden verarbeitet.


In [45]:
df = pd.read_csv("U:\Desktop\kmStand\csv-daten\DE-BB-002402_04_01.csv")
df

,Trip-Id,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentID,ZoneName,FRC,DeviceId,RawSpeed,RawSpeedMetric,CustomAttrs
0,fe40c1ade1f39c464a97d83433b78c03,0,2019-04-01T03:54:31.031Z,52.926330,12.804390,NaN,NaN,NaN,7526de2ec5c0aa700e63726c9461c424,29.0,kph,NaN
1,fe40c1ade1f39c464a97d83433b78c03,1,2019-04-01T03:55:33.033Z,52.923190,12.803140,NaN,NaN,NaN,7526de2ec5c0aa700e63726c9461c424,21.0,kph,NaN
2,fe40c1ade1f39c464a97d83433b78c03,2,2019-04-01T03:56:09.009Z,52.921110,12.800390,NaN,NaN,NaN,7526de2ec5c0aa700e63726c9461c424,1.0,kph,NaN
3,fe40c1ade1f39c464a97d83433b78c03,3,2019-04-01T03:56:32.032Z,52.919420,12.800560,NaN,NaN,NaN,7526de2ec5c0aa700e63726c9461c424,41.0,kph,NaN
4,fe40c1ade1f39c464a97d83433b78c03,4,2019-04-01T03:57:34.034Z,52.912080,12.801580,NaN,NaN,NaN,7526de2ec5c0aa700e63726c9461c424,50.0,kph,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
34138,0146f698467b5c0d983222cd2f49f632,171,2019-04-01T22:02:11.011Z,52.239395,11.016782,NaN,NaN,NaN,e72157777e535eff35b71820dfff70a2,89.0,kph,NaN
34139,0146f698467b5c0d983222cd2f49f632,172,2019-04-01T22:03:41.041Z,52.254631,10.996505,NaN,NaN,NaN,e72157777e535eff35b71820dfff70a2,87.0,kph,NaN
34140,0146f698467b5c0d983222cd2f49f632,173,2019-04-01T22:05:11.011Z,52.262166,10.986585,NaN,NaN,NaN,e72157777e535eff35b71820dfff70a2,66.0,kph,NaN
34141,0146f698467b5c0d983222cd2f49f632,174,2019-04-01T22:06:41.041Z,52.245150,10.991210,NaN,NaN,NaN,e72157777e535eff35b71820dfff70a2,89.0,kph,NaN


In [47]:
import os
import pandas as pd

def zaehle_trips(csv_verzeichnis, ergebnis_datei, tag="04_01"):
    daten = []

    # Durchsuche alle Dateien im Verzeichnis
    for datei in os.listdir(csv_verzeichnis):
        if datei.endswith(f"_{tag}.csv"):  # Nur Dateien für den angegebenen Tag
            dateipfad = os.path.join(csv_verzeichnis, datei)
            
            # Datei lesen und Trip-Ids zählen
            df = pd.read_csv(dateipfad)
            kennung = datei.split("_")[0]  # Extrahiere die "Kennung" aus dem Dateinamen
            anzahl_trips = df['Trip-Id'].nunique()  # Zählt die einzigartigen Trip-Ids
            
            # Speichere die Ergebnisse
            daten.append({"Kennung": kennung, "Anzahl Trips-Id": anzahl_trips})

    # Ergebnisse in ein DataFrame konvertieren und speichern
    ergebnis_df = pd.DataFrame(daten)
    ergebnis_dateipfad = os.path.join(csv_verzeichnis, ergebnis_datei)
    ergebnis_df.to_csv(ergebnis_dateipfad, index=False)
    
    print(f"Die Ergebnisse wurden erfolgreich unter {ergebnis_dateipfad} gespeichert.")

# Beispielverzeichnis und Ausgabedatei
csv_verzeichnis = r"U:\Desktop\kmStand\csv-daten"
ergebnis_datei = "ergebnisseRast_04_01.csv"
zaehle_trips(csv_verzeichnis, ergebnis_datei, tag="04_01")


Die Ergebnisse wurden erfolgreich unter U:\Desktop\kmStand\csv-daten\ergebnisseRast_04_01.csv gespeichert.


In [49]:
df = pd.read_csv("U:\Desktop\kmStand\csv-daten\ergebnisseRast_04_01.csv")
df

,Kennung,Anzahl Trips-Id
0,DE-BB-002402,103
1,DE-BB-274901,34
2,DE-BB-274902,29
3,DE-BB-283901,38
4,DE-BB-283902,35
...,...,...
127,DE-TH-009042,123
128,DE-TH-071031,18
129,DE-TH-071082,38
130,LKWaufBaden-Württemberg,2787


## Ergebnisse: 

In [2]:
import pandas as pd
import geopandas as gpd
import folium
from folium import Marker

# Definir el archivo CSV y el archivo GeoJSON
csv_file = r"U:\Desktop\kmStand\csv-daten\DE-BY-000030_04_01.csv"
geojson_file = r"U:\Desktop\kmStand\shapes\DE-BY-000030.geojson"

# Cargar los datos de los primeros 20 Trip-Id desde el archivo CSV
df = pd.read_csv(csv_file).head(1000)

# Crear el mapa centrado en el primer punto del CSV
mapa = folium.Map(location=[df['Latitude'].iloc[0], df['Longitude'].iloc[0]], zoom_start=10)

# Añadir los puntos de los Trip-Id al mapa
for _, row in df.iterrows():
    Marker([row['Latitude'], row['Longitude']], tooltip=f"Trip-Id: {row['Trip-Id']}").add_to(mapa)

# Cargar y añadir la carretera desde el archivo GeoJSON al mapa
carretera_gdf = gpd.read_file(geojson_file).to_crs("EPSG:4326")
for line in carretera_gdf.geometry:
    folium.GeoJson(line).add_to(mapa)

# Guardar y mostrar el mapa
output_map = r"U:\Desktop\kmStand\maps\DE-BY-000030_map.html"
mapa.save(output_map)
print(f"Mapa guardado en: {output_map}")


Mapa guardado en: U:\Desktop\kmStand\maps\DE-BY-000030_map.html
